In [1]:
from DefSD import *
from astropy.analytic_functions import blackbody
from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel



In [2]:
def finalFit(i, spec,lambd,ListLines,ListGal,spIvar,iterr, limResol):
    lineP = lProfile(i,spec,lambd,ListLines,ListGal, limResol)   #  Table of line profile indexes upper and lower 
    #linePaux=lineP['lambda'].data     
    #alslk = lineP['lambda'][0]
         #    it obtain the number of lines overlayed in a profile
    #for c in range(1,len(linePaux)):
    #    if (linePaux[c]==alslk) & (alslk!=0):
    #        linePaux[c-1] = 2
    #        linePaux[c] = 2
    #    else:
    #        linePaux[c] = 1
    #    alslk = lineP['lambda'][c]
        
    f1 = fitting.LevMarLSQFitter()
    lineStdDev = 3.5    #   default standard deviation
    Gaus=[]      #    array of models
    #VoigtModArr = []
    for x in range(len(ListLines)):       #     for each line profile it is created a gaussian model 
        if lineP['inf'][x]==0:            
            Gaus.append(models.Gaussian1D(amplitude=0,mean=ListLines['LAMBDA VAC ANG'][x],stddev=0))
            #VoigtModArr.append(models.Voigt1D(x_0=ListLines['LAMBDA VAC ANG'][x], amplitude_L=0, fwhm_L=0.0, fwhm_G=0.0))
                      #    it didn't model a profile for this line
        else:
                #v = np.where((lambd >= ListLines['LAMBDA VAC ANG'][x]-limResol) & (lambd <= ListLines['LAMBDA VAC ANG'][x]+limResol))
                      #    estimate the wavelenghts for the center of the line
                #innaaaads = v[0][0]
            innaaaads = lineP['lambda'][x]
            ampMax= ListGal[i][innaaaads]-spec[innaaaads]   #   max amplitude of the profile
            print lambd[innaaaads], ListGal[i][innaaaads], ampMax
            nLines = 1 #linePaux[x]                        #   obtain the number of overlaying line profiles 
            try:
                f = FWHM(lambd[lineP['inf'][x]:lineP['sup'][x]], ListGal[i][lineP['inf'][x]:lineP['sup'][x]])
                          #    calculate the FWHM of the profile
                while len(f)==0:
                    f = FWHM(lambd[(lineP['inf'][x]-coasi):(lineP['sup'][x]+coasi)], ListGal[i][(lineP['inf'][x]-coasi):(lineP['sup'][x]+coasi)])
                           #  calculate FWHM so often as it is found a non profile spectrum
                print f
                stdLine = (1/2.3548)*f[0]/nLines     #   estimate the observed standard deviation
            except:
                stdLine = (1/2.3548)*lineStdDev/nLines
            lineWidthBase = (lambd[lineP['sup'][x]] - lambd[lineP['inf'][x]])/(2*nLines)
                      ##   calculate the observed line width at the base of the profile
            Gaus.append(models.Gaussian1D(amplitude=ampMax,mean=ListLines['LAMBDA VAC ANG'][x],stddev=stdLine, bounds={'mean':(ListLines['LAMBDA VAC ANG'][x]-lineWidthBase/(2*nLines), ListLines['LAMBDA VAC ANG'][x]+lineWidthBase/(2*nLines)),'stddev':(0.8*stdLine, 1.5*stdLine)}))
            #VoigtModArr.append(models.Voigt1D(x_0=ListLines['LAMBDA VAC ANG'][x], amplitude_L=ampMax, fwhm_L=stdLine, fwhm_G=stdLine))
    sum_Gaussian=Gaus[0]
    for ooasia in range(1,len(Gaus)):
            #  sum of all the models respective to all the wavelenghts 
        sum_Gaussian=sum_Gaussian + Gaus[ooasia]
    gaussian_fit = f1(sum_Gaussian, lambd, ListGal[i]-spec, maxiter=iterr)
    return gaussian_fit, lineP, spec     ##  return the fitting to both models and line profiles wavelenghts

    #sum_Voigt=VoigtModArr[0]
    #for ooasia in range(1,len(VoigtModArr)):
            #  sum of all the models respective to all the wavelenghts 
    #    sum_Voigt=sum_Voigt + VoigtModArr[ooasia]
    #voigt_fit = f1(sum_Voigt, lambd, ListGal[i]-spec, maxiter=iterr)
    #return voigt_fit, lineP, spec     ##  return the fitting to both models and line profiles wavelenghts


In [3]:
jj=0
resultListDT=Table.read('newObjSDSSFound.tbl', format='ascii')
#  newObjSDSSFound
#  dataArt1SDSSFound
name = resultListDT['ObjID'][0]
z = 0#resultListDT['z'][0]
hdulist = fits.open('Exam_fits/'+name+'.fits')
head= hdulist[0].header
total= hdulist[1].data
linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
lambd = linlam/(1+z)
spectraFlux = total['flux']
Spectra = []
Spectra.append(spectraFlux)
Lambda=[]
Lambda.append(lambd)
ListaLineas=Table.read('LinesStar.txt', format='ascii')
#   LinesAll
#  LinesStar
spectraIvar = total['ivar']
SpectraIvar = []
SpectraIvar.append(spectraIvar)
limitR = Lambda[jj][1]-Lambda[jj][0]

In [4]:
def smoothSpectrum(spectrum, factorStdKernel, divisi):
    jj = 0
    if divisi == 1:
        stdSpDiv = np.std(spectrum)
        gauss_kernel = Gaussian1DKernel(factorStdKernel*stdSpDiv)
        smoothed_data_gauss = convolve(spectrum, gauss_kernel, boundary=u'extend', normalize_kernel=True)
        spssp = smoothed_data_gauss[:]
    else:
        spssp = [100]*len(spectrum)
        lambdDiv = int(mt.ceil(len(spssp)/divisi))
        lambdDivArr = [lambdDiv*a for a in range(divisi)]
        lambdDivArr[-1] = len(spectrum)
        SG = []
        for aa in range(len(lambdDivArr)-1):
            stdSpDiv = np.std(spectrum[lambdDivArr[aa]:lambdDivArr[aa+1]])
            #box_kernel = Box1DKernel(stdSpDiv)
            #smoothed_data_box = convolve(Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]], box_kernel)
            #print stdSpDiv
            gauss_kernel = Gaussian1DKernel(factorStdKernel*stdSpDiv)
            meanSpDiv = np.mean(spectrum[lambdDivArr[aa]:lambdDivArr[aa+1]])
            smoothed_data_gauss = convolve(spectrum[lambdDivArr[aa]:lambdDivArr[aa+1]], gauss_kernel, boundary=u'extend',fill_value=meanSpDiv, normalize_kernel=True)
            SG.append(smoothed_data_gauss)
            spssp[lambdDivArr[aa]:lambdDivArr[aa+1]] = smoothed_data_gauss # smoothed_data_box
    return spssp
    #print "std: ", , "err: ", np.mean(SpectraIvar[0][lambdDivArr[aa]:lambdDivArr[aa+1]])
    #print Spectra[0][lambdDivArr[aa]:lambdDivArr[aa+1]]

#stdSp = np.std(Spectra[0])
#STD = [stdSp]*len(Spectra[0])
#spssp = spssp+STD

In [5]:
def lProfile(i,spec, lambd,ListLines,ListGal, limResoluc):
    linePr = Table(names=('lambda', 'inf', 'sup', 'Cinf', 'Csup'), dtype=('i5','i5', 'i5','i5', 'i5'))
    specRedLines = smoothSpectrum(ListGal[i], 1, 1)

    for c in range(0, len(ListLines)):
        lambdLine=ListLines['LAMBDA VAC ANG'][c]
        v = np.where((lambd >= lambdLine-8*limResoluc) & (lambd <= lambdLine+8*limResoluc))
        l_inf = v[0][0]
        l_sup = v[0][-1]
        # determinar el maximo valor en el espectro en el intervalo
        # inf sup y compararlo con el del contrinuo en el mismo intervalo,
        # que nos indica si es un perfil de emision o de absorcion
        
        #indMaxInt = np.where(ListGal[i][l_inf:l_sup]==np.max(ListGal[i][l_inf:l_sup]))
        #if (ListGal[i][indMaxInt] - specRedLines[indMaxInt]) > 0:
        
        if np.mean(ListGal[i][l_inf:l_sup]-specRedLines[l_inf:l_sup]) > 0:
            lEmiss = True
        else:
            lEmiss = False
        print lEmiss
        #if (np.mean(ListGal[i][l_inf:(l_sup + 1)]) - np.mean(spec[l_inf:(l_sup + 1)])) > 0:
        #    lEmiss = True
        #else:
        #    lEmiss = False
        difIntAux = ListGal[i][l_inf + 1]-ListGal[i][l_inf]
        interv = []
        interv.append(l_inf)

        for ss in range(l_inf,l_sup, 1):
            difInt = ListGal[i][ss + 1]-ListGal[i][ss]
            #print ss, difInt, difIntAux
            if difIntAux*difInt < 0.0:
                interv.append(ss)
                #print "si", ss
            difIntAux = ListGal[i][ss + 1]-ListGal[i][ss]

        interv.append(l_sup)
        # aqui ya se tiene los intervalos donde hay minimos o maximos
        # entonces se busca done estan los maximos valores mas cercanos a lambda teorica
        print interv
        #buscar lineas de emision
        #if lEmiss == True:
        lll_inf = interv[0]
        if (interv[1]-interv[0]) >= 0:
            ll_inf = interv[0]
            startIndex = 2
        else:
            ll_inf = interv[1]
            startIndex = 3
        #else:
        #    lll_inf = interv[0]
        #    if (interv[1]-interv[0]) <= 0:
        #        ll_inf = interv[0]
        #        startIndex = 2
        #    else:
        #        ll_inf = interv[1]
        #        startIndex = 3
        deltaLambd = 100
        minIndex = 0
        line_min = 0
        line_max = 0
        for issi in range(startIndex, len(interv), 2):
            if lEmiss == True:
                indMaxInt = np.where(ListGal[i][ll_inf:interv[issi]+1]==np.max(ListGal[i][ll_inf:interv[issi]+1]))
            else:
                indMaxInt = np.where(ListGal[i][ll_inf:interv[issi]+1]==np.min(ListGal[i][ll_inf:interv[issi]+1]))
            indexMax = indMaxInt[0][0]+lll_inf
            if mt.fabs(lambdLine - lambd[indexMax]) < deltaLambd:
                minIndex = indexMax
                line_min = ll_inf
                line_max = interv[issi]
        #-------------
        widthLine = 4
        widthLineCont = 2
        lDel = (l_sup-l_inf)*widthLineCont
        infSTD = np.std(ListGal[i][(l_inf-widthLine*lDel):l_inf])
        supSTD = np.std(ListGal[i][l_sup:(l_sup+widthLine*lDel)])
        
        while (3*infSTD < mt.fabs(ListGal[i][l_inf]-specRedLines[l_inf])):
            l_inf = l_inf - 1
        while (supSTD < mt.fabs(ListGal[i][l_sup]-specRedLines[l_sup])):
            l_sup = l_sup + 1
            
        #------------
        if (minIndex == 0) or (mt.fabs(lambdLine - lambd[minIndex]) >= 4.0):
            v = np.where((lambd >= lambdLine-4*limResoluc) & (lambd <= lambdLine+4*limResoluc))
            if (len(v[0])==0) or ((v[0][-1]-v[0][0]) <= 1):
                minIndex = 0
                line_min = 0
                line_max = 0
            else:  # (v[0][-1]-v[0][0]) > 1:
                minIndex = v[0][0] + (v[0][-1]-v[0][0])/2
                line_min = v[0][0]
                line_max = v[0][-1]
                print "NO se encontró un pico, por defecto se ubica en: ", lambd[minIndex]," lambda teórica: ", lambdLine
        else:   # if (lambdLine - lambd[minIndex]) < 2.0:
            print "se encontró el pico más cercano en : ",  lambd[minIndex], "con lambda teorica en :" , lambdLine
            ind = 1
            l_inf = minIndex-ind
            l_sup = minIndex+ind
            initVal = np.mean(ListGal[i][l_inf:l_sup])
            slopeRight = spec[l_sup+1]-spec[l_sup]
            cooc = 0
            while cooc<1:
                l_sup=l_sup+1
                sloRaux = slopeRight
                slopeRight = spec[l_sup+1]-spec[l_sup]
                if slopeRight*sloRaux <= 0:
                    cooc = cooc + 1
            cooc = 0
            slopeLeft = spec[l_inf-1]-spec[l_inf]
            while cooc<1:
                l_inf=l_inf-1
                sloLaux = slopeLeft
                slopeLeft = spec[l_inf-1]-spec[l_inf]
                if slopeLeft*sloLaux <= 0:
                    cooc = cooc + 1
            line_min = l_inf + 0
            line_max = l_sup + 0
        linePr.add_row((minIndex, line_min, line_max, 0,0))
    return linePr

In [6]:
spssp = smoothSpectrum(Spectra[jj], 0.01, 1)
kLineP = lProfile(0, spssp, Lambda[0], ListaLineas, Spectra, limitR)
kLineP

False
[1876, 1881, 1883, 1885, 1886]
se encontró el pico más cercano en :  5892.51 con lambda teorica en : 5892.53
False
[3469, 3473, 3475]
se encontró el pico más cercano en :  8501.59 con lambda teorica en : 8500.0
False
[3492, 3495, 3498]
se encontró el pico más cercano en :  8544.76 con lambda teorica en : 8544.9
False
[3553, 3556, 3559]
se encontró el pico más cercano en :  8665.63 con lambda teorica en : 8665.01
False
[2301, 2305, 2309]
se encontró el pico más cercano en :  6496.81 con lambda teorica en : 6496.36
False
[2346, 2348, 2351, 2354]
se encontró el pico más cercano en :  6565.99 con lambda teorica en : 6563.96
False
[1648, 1649, 1652, 1655, 1657, 1658]
se encontró el pico más cercano en :  5589.85 con lambda teorica en : 5590.74


lambda,inf,sup,Cinf,Csup
int32,int32,int32,int32,int32
1881,1866,1883,0,0
3473,3469,3477,0,0
3495,3489,3501,0,0
3556,3552,3560,0,0
2305,2301,2312,0,0
2351,2348,2354,0,0
1652,1649,1656,0,0


In [7]:
#  Nuevo  encuentra el continuo a partir del error en cada medida

spec2 = spssp

specRedLines = smoothSpectrum(Spectra[jj], 1, 1)

IPP = []
SPP = []
for ina in range(len(kLineP)):
    widthLine = 4
    widthLineCont = 3
    #lInf = kLineP[ina]['Cinf']
    #lSup = kLineP[ina]['Csup']
    lInf = kLineP[ina]['inf']
    lSup = kLineP[ina]['sup']
    lDel = (lSup-lInf)*widthLineCont
    
    INFInterval=[]
    SUPInterval=[]
    
    if (lInf-widthLine*lDel) >= 0 :
        lInf = lInf-widthLine*lDel
        for ioas in range(lInf, (lInf+lDel), 1):
            if mt.fabs(Spectra[jj][ioas]-specRedLines[ioas]) < 3*SpectraIvar[jj][ioas]:
                INFInterval.append(Spectra[jj][ioas])
        INFIntMean = np.mean(INFInterval)
    
    
    if ((lInf-widthLine*lDel) < 0) or (INFInterval == []):
        lInf = kLineP[ina]['inf']
        INFIntMean = Spectra[jj][lInf]
        #INFInterval.append(Spectra[jj][lInf])
    
    if (lSup+widthLine*lDel) <= len(lambd):
        lSup = lSup+widthLine*lDel
        for ioas in range((lSup-lDel), lSup, 1):
            if mt.fabs(Spectra[jj][ioas]-specRedLines[ioas]) < 3*SpectraIvar[jj][ioas]:
                SUPInterval.append(Spectra[jj][ioas])
        SUPIntMean = np.mean(SUPInterval)
    if ((lSup+widthLine*lDel) > len(lambd)) or (SUPInterval == []):
        lSup = kLineP[ina]['sup']
        SUPIntMean = Spectra[jj][lSup]
        #INFInterval.append(Spectra[jj][lSup])
    
    
    m, b = linModel( Lambda[jj][lInf] , INFIntMean , Lambda[jj][lSup], SUPIntMean)
    lineLambd = Lambda[jj][lInf:lSup+1]
    lineIntCont = lines( lineLambd , m, b)
    spec2[lInf:lSup+1] = lineIntCont
    #print "linea ", ina, "len(linea)= ", Lambda[jj][lInf] , INFIntMean , Lambda[jj][lSup], SUPIntMean
    #spec2[lInf:lSup] = SspecRedLi
    
    #IPP.append(specRedLines[(lInf-widthLineCont*lDel):lInf])
    #SPP.append(specRedLines[lSup:(lSup+widthLineCont*lDel)])
    #spec2[(lInf-widthLineCont*lDel):lInf] = IPP[ina]
    #spec2[lSup:(lSup+widthLineCont*lDel)] = SPP[ina]

SpectraCont=[]
SpectraCont.append(spec2)

/usr/lib64/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/lib64/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:

%matplotlib

specRedLines1 = smoothSpectrum(Spectra[jj], 1, 1)

plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
plt.plot(Lambda[jj],specRedLines1, lw=2, c = 'cyan', label='Spectra')
#plt.plot(lamandaS,specRedLiS, lw=2, c = 'cyan', label='Spectra')
#plt.plot(Slamanda,SspecRedLi, lw=2, c = 'cyan', label='Spectra')
lPPP = kLineP
#plt.plot(Lambda[jj],Spectra[jj]-spssp, lw=0.5, c = 'gray', label='Spectra')
for iasiao in range(len(lPPP)):
    plt.plot(Lambda[jj][lPPP['inf'][iasiao]:(lPPP['sup'][iasiao]+1)],Spectra[jj][lPPP['inf'][iasiao]:(lPPP['sup'][iasiao]+1)], lw=2.0, c = 'b')
#    plt.scatter(Lambda[jj][lPPP['sup']],Spectra[jj][lPPP['sup']], lw=2.0, c = 'cyan', label='Spectra')
#plt.plot(Lambda[jj],specRedLines, lw=2, c = 'cyan', label='Spectra')


Using matplotlib backend: Qt4Agg


In [63]:
Model, lineProf, spec2 = finalFit(0, SpectraCont[jj], Lambda[jj],ListaLineas,Spectra, spectraIvar,2000, limitR)
Modelos = []
Modelos.append(Model)
LS=np.zeros([len(ListaLineas),10])
Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth', 'S/N>3'),
                    dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8','S5'))
Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
Tabla['Element']=ListaLineas['SPECTRUM']

AA=split_list(Model.parameters, wanted_parts=len(ListaLineas))
for ii in range(len(ListaLineas)):
    if 3*mt.fabs(np.mean(spectraIvar[lineProf[0]['inf']:lineProf[0]['sup']])) >= mt.fabs(AA[ii][0]):
        Tabla['S/N>3'][ii] = str(False)
    else:
        Tabla['S/N>3'][ii] = str(True)
    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], AA[ii][2]])
Tabla

False
[3469, 3473, 3475]
se encontró el pico más cercano en :  8501.59 con lambda teorica en : 8500.0
False
[3492, 3495, 3498]
se encontró el pico más cercano en :  8544.76 con lambda teorica en : 8544.9
False
[3553, 3556, 3559]
se encontró el pico más cercano en :  8665.63 con lambda teorica en : 8665.01
False
[2301, 2305, 2309]
se encontró el pico más cercano en :  6496.81 con lambda teorica en : 6496.36
False
[2346, 2348, 2351, 2354]
se encontró el pico más cercano en :  6565.99 con lambda teorica en : 6563.96
False
[1648, 1649, 1652, 1655, 1657, 1658]
se encontró el pico más cercano en :  5589.85 con lambda teorica en : 5590.74
8501.59 174.648 -58.5282
[-5.86914062]
8544.76 139.688 -85.7042
[-5.89941406]
8665.63 138.531 -86.0954
[-5.99121094]
6496.81 213.635 -58.1209
[-7.47949219 -4.53320312]
6565.99 218.694 -44.363
[-7.47949219 -4.53320312]
5589.85 183.493 -38.5414


Amplitude,Wavelength,Sigma,Lambda Vac,Element,Flux,Flux_Err,FWHM,eqWidth,S/N>3
float64,float64,float64,float64,string64,float64,float64,float64,float64,string40
-41.1580400361,8501.13189639,-3.73862363577,8500.0,[O I],0.0,0.0,0.0,0.0,True
-73.4713544245,8545.30059274,-3.75790771775,8544.9,[O I],0.0,0.0,0.0,0.0,True
-62.865298627,8665.42412,-3.81638203085,8665.01,[O I],0.0,0.0,0.0,0.0,True
-42.6708458618,6498.66834107,-4.76441238375,6496.36,[O I],0.0,0.0,0.0,0.0,True
-24.3179615508,6565.45002249,-4.76441238375,6563.96,[O I],0.0,0.0,0.0,0.0,True
-42.296523816,5590.36024192,1.42531701614,5590.74,[O I],0.0,0.0,0.0,0.0,True


In [64]:
%matplotlib
plt.plot(Lambda[jj], Spectra[jj]-SpectraCont[jj], c='k', lw=0.8, label='Model')
plt.plot(Lambda[jj], Modelos[jj](Lambda[jj]), c='cyan', lw=1)


Using matplotlib backend: Qt4Agg


In [13]:
#    Diferencia de longitudes de onda teoricas y medidas
for oioo in range(len(kLineP)):
    print Lambda[jj][kLineP['lambda'][oioo]]-ListaLineas['LAMBDA VAC ANG'][oioo]

3.7156171875
0.9326171875
1.44344140625
1.75687109375
1.4214921875
2.57619921875
1.0887890625
1.35912109375
-0.47828125
0.80375
2.58798828125
2.9300390625
1.9436328125
2.225625
1.78703125
3.21327734375


In [11]:
head

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TAI     =        4561174190.76 / 1st row - Number of seconds since Nov 17 1858  
RA      =            253.87206 / 1st row - Right ascension of telescope boresigh
DEC     =            32.014022 / 1st row - Declination of telescope boresight (d
EQUINOX =              2000.00 /                                                
RADECSYS= 'FK5     '           /                                                
TAIHMS  = '08:36:47.81'        / 1st row - TAI time (HH:MM:SS.SS) (TAI-UT = appr
TIMESYS = 'tai     '           / TAI, not UTC                                   
MJD     =                52791 / MJD of observation                             
MJDLIST = '52791   '        